# Churn prediction

**Customer churn/attrition** -  **the percentage of customers that stop using a company's products or services.**
It is one of the most important metrics for a business, as it usually costs more to acquire new customers than it does to retain existing ones.

By using Survival Analysis, not only companies can predict if customers are likely to stop doing business but also when that event might happen.

based on the tutorial https://square.github.io/pysurvival/tutorials/churn.html

## Data
Today, we will try to predict whether a customer will change telecommunications provider called Telco, data can be loaded here: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

The dataset includes information about:

+ Customers who left within the last month – the column is called Churn
+ Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
+ Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
+ Demographic info about customers – gender, age range, and if they have partners and dependents

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd

data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
data.head()

## Data Preparation

+ Selecting necessary columns
+ One-hot encoding
+ Checking NAs

In [ ]:
# no duplicates
data.duplicated().sum()

In [ ]:
# we do not need customerID
data = data.drop(columns=['customerID'], axis=1)

In [ ]:
data.head()

In [ ]:
# no NAs :)
data.isna().sum()

In [ ]:
for col in data.columns:
    print(f"{col}:")
    print(data[col].value_counts())
    print()

In [ ]:
col_binary = ['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
col_cat = ['MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

In [ ]:
# 0 - No, 1 - Yes
data[col_binary] = data[col_binary].applymap(lambda x: 0 if x == "No" else 1)
# 0 -Male, 1 - Female
data['gender'] = data['gender'].apply(lambda x: 0 if x == "Male" else 1)

In [ ]:
data.head()

In [ ]:
# one-hot encoding for categorical with more than 2 categories
pd.get_dummies(data[col_cat], drop_first=True)

In [ ]:
numerical = [x for x in data.columns if x not in col_binary and x not in col_cat and x != 'gender']

In [ ]:
data[numerical].dtypes

In [ ]:
for i in range(len(data)):
    try:
        x = float(data['TotalCharges'][i])
    except ValueError:
        print(i,data['TotalCharges'][i])

In [ ]:
data = pd.concat([data['gender'], data[col_binary], pd.get_dummies(data[col_cat], drop_first=True), data[numerical]], axis=1)

In [ ]:
data.columns

In [ ]:
data = data[data['TotalCharges'] != ' ']

In [ ]:
data['TotalCharges'] = data['TotalCharges'].astype(float)

In [ ]:
data[numerical].dtypes

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data

In [ ]:
data.drop('index', axis=1, inplace=True)

## Preparations for modelling

Let's split our data to train and test. We will train our model on train, and then calculate metrics on test for model evaluation.


**NOTE:** If you also want to tune some hyperparameters of the model, you need to split your data to 3 parts: train, validation and test. So, you train all your models on train, compare the metrics on validation set for different parameters, choose the best parameters according to the validation set. And after that test the final model on test set. This will help to avoid overfitting.

In [ ]:
from sklearn.model_selection import train_test_split

N = data.shape[0]
index_train, index_test = train_test_split(range(N), test_size = 0.35)
data_train = data.loc[index_train].reset_index(drop = True)
data_test  = data.loc[index_test].reset_index(drop = True)

features = [x for x in data.columns if x != 'Churn' and x != 'tenure']

In [ ]:
# RandomSurvivalForest needs y as the array in som structured way
# list with pairs (churn, tenure)
churn_tenure_train = [(e1,e2) for e1,e2 in np.array(data_train[["Churn", "tenure"]])]

# to numpy array
churn_tenure_train = np.array(churn_tenure_train, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])


# the same for test

churn_tenure_test = [(e1,e2) for e1,e2 in np.array(data_test[["Churn", "tenure"]])]
churn_tenure_test = np.array(churn_tenure_test, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

## Modelling


### Random Survival Forest 

Article about this method: https://projecteuclid.org/journals/annals-of-applied-statistics/volume-2/issue-3/Random-survival-forests/10.1214/08-AOAS169.full

Shorter article: https://www.randomforestsrc.org/articles/survival.html


tutorial: https://scikit-survival.readthedocs.io/en/stable/user_guide/random-survival-forest.html



Works with the right-censored data (if only the lower limit l for the true event time T is known such that T > l, this is called right censoring. Right censoring will occur, for example, for those subjects whose birth date is known but who are still alive when they are lost to follow-up or when the study ends.)


As in the classical Random Forest, here in each tree the node is separated into two nodes using some criteria. In Random Forest, it is usually Gini coefficient or entropy (for classification). Here we use different function. A node maximizes survival difference between its children. 

It uses the Nelson–Aalen estimator (as the cumulative hazard function)

In [ ]:
#!pip install scikit-survival
#there can be an error about the conflict with scikit-learn package
#! pip install -U scikit-learn==1.1.0

In [ ]:
from sksurv.ensemble import RandomSurvivalForest

In [ ]:
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           n_jobs=-1,
                           random_state=10)
rsf.fit(data_train[features], churn_tenure_train)

## Evaluating on test data

(from the original article) **Prediction error**. To estimate prediction error, we use Harrell’s concordance index [Harrell et al. (1982)]. The C-index (concordance index) is related
to the area under the ROC curve [Heagerty and Zheng (2005)]. 

It estimates the
**probability that, in a randomly selected pair of cases, the case that fails first had a
worst predicted outcome.** The interpretation of the C-index as a misclassification
probability is attractive, and is one reason we use it for prediction error. Another attractive feature is that, unlike other measures of survival performance, the C-index
does not depend on a single fixed time for evaluation. The C-index also specifically
accounts for censoring.



Prediction error = 1 - Concordance index

If Prediction error == Concordance index == 0.5 5 indicates prediction no better than random guessing.

In [ ]:
# a concordance index 
rsf.score(data_test[features], churn_tenure_test)

## Predicting

For prediction, a sample is dropped down each tree in the forest until it reaches a terminal node. Data in each terminal is used to non-parametrically estimate the survival and cumulative hazard function using the Kaplan-Meier and Nelson-Aalen estimator, respectively. In addition, a risk score can be computed that represents the expected number of events for one particular terminal node. The ensemble prediction is simply the average across all trees in the forest.

Let’s predict churn for a several customers from test data

In [ ]:
test_customers = data.sample(n=5, random_state=10)
test_customers

Algorithm predicts a risk score for each observation (in the article it's called mortality).

A risk score represents the expected number of events for one particular terminal node. This estimates **the number of deaths expected if all cases were similar to X** (so all observations go to this node). 

We can see at this number as a risk score of churn - the bigger the value, the more the risk that this customer will leave.

In [ ]:
pd.Series(rsf.predict(test_customers[features]))

In [ ]:
plt.hist(rsf.predict(data_test[features]), bins=30)

We can have a more detailed insight by considering the **predicted survival function**. 


$$S(t|{\bf X})=\mathbb{P}\{T^o> t|{\bf X}\}$$ $T^o$ - survival event time

So, this is the probablity that at a given time t, the person is alive (death would be at time $T^o$ which is bigger then t)

In [ ]:
surv = rsf.predict_survival_function(test_customers[features], return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.event_times_, s, where="post", label=str(i))
plt.ylabel("Survival probability")
plt.xlabel("Time")
plt.legend()
plt.grid(True)

Alternatively, we can also plot the predicted **cumulative hazard function.**


$$F(u|{\bf X})=\mathbb{P}\{T^o\le u|{\bf X}\}.$$


where $u$ is some time value.

The probablity, that a person will not survive after time $u$ (a person will die at time $u$ or earlier)


The above two functions are calculated in the forest with summing up the values of deaths/individuals at risk for each node/

In [ ]:
surv = rsf.predict_cumulative_hazard_function(test_customers[features], return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.event_times_, s, where="post", label=str(i))
plt.ylabel("Cumulative hazard")
plt.xlabel("Time")
plt.legend()
plt.grid(True)

## Feature Importance

Here we use permutation-based feature importance.

Permutation feature importance is a model inspection technique that can be used for any fitted estimator when the data is tabular. This is especially useful for non-linear or opaque estimators. The permutation feature importance is defined to be **the decrease in a model score when a single feature value is randomly shuffled**. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. This technique benefits from being model agnostic and can be calculated many times with different permutations of the feature.

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    rsf, data_test[features], churn_tenure_test, n_repeats=15, random_state=10)

pd.DataFrame(
    {k: result[k] for k in ("importances_mean", "importances_std",)},
    index=data_test[features].columns).sort_values(by="importances_mean", ascending=False)

## Task for you

**Deadline**: 04.10.2022 12:00, my e-mail: aspestova@hse.ru.

!!! Send your solution in **html** format, please






1. Take any other model suitable for Survival Analysis. For example, it can be from **slkearn-survival** package (https://scikit-survival.readthedocs.io/en/stable/user_guide/index.html) or **pysurvival** (the example of a function with the link to the documentation is given below). In a couple of words describe how the model differ from the model we used on the practice.

2. Train the model on the train data. Compare the model results with those obtained on the practice (by concordance index).

(You can use default parameters  or to tune the hyperparameters, but if you do so, split the data into 3 parts. And compare your final model and the model from the practice only on test set, tuning of hyperparameters should be done in validation set!)

**Note**: if you use some other package (not sklearn) and it doesn't calculate the concordance index for you, then you can use the function from sklearn: https://scikit-survival.readthedocs.io/en/stable/api/generated/sksurv.metrics.concordance_index_censored.html. The first ans the second arguments are churn and tenure from test data, the third one - predictions of risk score by your model. **pysurvival** has method for calculation this index!

3. Compute feature importance. **pysurvival** also can do it. Compare the results with the feature importances from the model we built on the practice. 

### Conditional Survival Forest 

**The Conditional Survival Forest model** was developed by [Wright et al. in 2017](https://arxiv.org/pdf/1605.03391.pdf) to improve the Random Survival Forest training, whose objective function tends to favor splitting variables with many possible split points.

Tutorial on it: https://square.github.io/pysurvival/tutorials/churn.html

You can use this library and this model (or some other if you find it)